In [2]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
#df = pd.read_csv("Inputs1stGen.csv", low_memory = False)
df = pd.read_csv("Inputs1stGen.csv", low_memory = False, nrows = 100000)
df = df.dropna()

In [4]:
# W&B Integration
import wandb
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="clemha-mli", # replace this to your own W&B account
    # Set the wandb project where this run will be logged.
    project="HackerNews Prediction from HyperParameterHippies",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": 0.001,
        "architecture": "Our architecture",
        "dataset": "Hackernews text and a lot",
        "epochs": 30,
    })

wandb: Currently logged in as: clemha (clemha-mli) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
from datetime import datetime
from urllib.parse import urlparse

'''
The following section gives us all the parameters relating to time:
1. Day of week
2. Time of day
3. Month of Year
4. Year of past 
'''

#Converting UNIX timestamp to datetime
df['datetime'] = pd.to_datetime(df['time'])

#Getting the names of the days of the week
df['day_of_week'] = df['datetime'].dt.day_name()

#Creating a key of days of the week to numbers
day_map = {
    'Monday': 0, 'Tuesday': 1, 'Wednesday':2, 'Thursday':3, 'Friday':4, 'Saturday':5, 'Sunday':6
}

#Assigning numbers to the days of the week corresponding to the dataset
df["day_of_week_num"] = df['day_of_week'].map(day_map)

#Getting the names of the months of year
df['month'] = df['datetime'].dt.month_name()

#Creating a key of months of the year to numbers
month_map = {
    'January': 1, 'February': 2, 'March': 3, 'April':4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October':10, 'November': 11, 'December': 12
}

df['month_num'] = df['month'].map(month_map)

#Getting the hour of the post
df['hour'] = df['datetime'].dt.hour

#Getting the year of the post
df['year'] = df['datetime'].dt.year


In [6]:
print(df)

                   by   karma  \
0         ColinWright  127765   
1                hncj      44   
2          andymboyle     714   
3             digisth    2395   
4             jazzdev     543   
...               ...     ...   
99995  ahmetalpbalkan      97   
99996          cocoon      49   
99997   thestartupguy       5   
99998         ph0rque   23296   
99999        cshekhar     117   

                                                   title  \
0      "What May Happen in the Next Hundred Years", f...   
1           Getting Started with JavaScript Unit Testing   
2      Armstrong, the Django-based and open-source ne...   
3                   Why Web Reviewers Make Up Bad Things   
4      You Weren't Meant to Have a Boss: The Cliff Notes   
...                                                  ...   
99995  Bookmarklet: Hide MG Siegler's posts on TechCr...   
99996           Who owns the right to analyze your data?   
99997  Freelancer or Entrepreneur? Seth Godin cares a...   
999

In [7]:
max_time = df['datetime'].max()
print(max_time)

2024-02-29 16:59:00


In [8]:
df['time_since_post1'] = max_time - df['datetime']

In [9]:
print(df['time_since_post1'])

0       4511 days 00:32:00
1       4420 days 05:19:35
2       4511 days 00:31:24
3       3880 days 11:42:34
4       5814 days 07:12:35
               ...        
99995   4601 days 18:50:23
99996   4601 days 18:49:21
99997   3907 days 06:05:59
99998   4629 days 07:20:50
99999   4629 days 07:18:03
Name: time_since_post1, Length: 93708, dtype: timedelta64[ns]


In [10]:
from lib8 import Word2Vec
import torch

embedding_dim = 50
device_ids = [0, 1] # Use the first two GPUs
device = torch.device(f"cuda:{device_ids[0]}" if torch.cuda.is_available() else "cpu")

# --- Initialize Model, Loss, and Optimizer ---
MODEL_PATH = f"w2v/word2vec_pytorch_{embedding_dim}.pth"

model = Word2Vec.load_from_checkpoint(MODEL_PATH, device)
model.eval()

--- Switched to eval mode. Pre-warming cache... ---
--- Calculating and caching normalized embeddings... ---


Word2Vec(
  (in_embeddings): Embedding(71290, 50)
  (out_embeddings): Linear(in_features=50, out_features=71290, bias=True)
)

In [11]:
similar_words = model.find_most_similar('germany', top_n=10)
# similar_words = model.find_most_similar('computer', top_n=10)

# p = model.get_vector('paris')
# f = model.get_vector('france')
# g = model.get_vector('germany')

f = model.get_sentence_vector('Elon Musk is alive')

# bgf = p - f + g
print(f)
similar_words = model.find_most_similar_by_vector(f, top_n=10)

if similar_words:
    print("-" * 30)
    for word, score in similar_words:
        print(f"{word:<20} {score:.4f}")
    print("-" * 30)

--- Using cached embeddings. ---
tensor([ 0.0151, -0.2932,  0.0731, -0.0389, -0.0133,  0.0778,  0.2062,  0.0685,
         0.1959,  0.2224,  0.2976,  0.0808,  0.0589, -0.2884,  0.0526, -0.1409,
        -0.0912, -0.1140, -0.1360, -0.2357,  0.3193, -0.0581,  0.0759, -0.0365,
        -0.1061, -0.0400,  0.1124, -0.1369, -0.0608, -0.0191, -0.1755,  0.2196,
         0.1463, -0.1008, -0.0974,  0.0120, -0.1018,  0.0551, -0.0144,  0.1746,
         0.0509, -0.1130, -0.2061, -0.0268,  0.1000, -0.1205, -0.1274,  0.0267,
         0.2003, -0.0287])
--- Using cached embeddings. ---
------------------------------
is                   0.7491
alive                0.6933
was                  0.5683
punchline            0.5467
happens              0.5366
isn                  0.5364
stands               0.5295
lop                  0.5261
makes                0.5194
distasteful          0.5169
------------------------------


In [12]:
'''
The following code gives us:
2. Time since the post was posted
'''

df['time_since_post'] = df['time_since_post1'].dt.total_seconds()

In [13]:
print(df['time_since_post'])

0        389752320.0
1        381907175.0
2        389752284.0
3        335274154.0
4        502355555.0
            ...     
99995    397594223.0
99996    397594161.0
99997    337586759.0
99998    399972050.0
99999    399971883.0
Name: time_since_post, Length: 93708, dtype: float64


In [14]:
'''
The following code gives us:
1. URL
2. Domain
'''

from urllib.parse import urlparse
import pandas as pd
from tqdm import tqdm

# Enable tqdm for pandas apply
tqdm.pandas(desc="Parsing URLs")

# Define a safe parser
def safe_urlparse(url):
    try: 
        parsed = urlparse(url)
        return parsed.netloc, parsed.path
    except Exception:
        return '', ''

# Ensure URL column is clean
df['url'] = df['url'].fillna('').astype(str)

# Apply with progress bar
df[['domain_name', 'url_path']] = df['url'].progress_apply(
    lambda u: pd.Series(safe_urlparse(u))
)

Parsing URLs: 100%|██████████████████| 93708/93708 [00:02<00:00, 41006.01it/s]


In [15]:
print(safe_urlparse('https://chatgpt.com/c/684813b3-5bfc-8013-8f49e'))

('chatgpt.com', '/c/684813b3-5bfc-8013-8f49e')


In [16]:
print(df['url'])

0        http://www.howtobearetronaut.com/wp-content/up...
1        http://blogs.lessthandot.com/index.php/WebDev/...
2        http://www.marketwatch.com/story/the-bay-citiz...
3        http://bits.blogs.nytimes.com/2013/07/15/why-w...
4                     http://paulgraham.com/bossnotes.html
                               ...                        
99995                                 http://rpa.tl/hidemg
99996    http://blog.getcocoon.com/2011/07/25/who-owns-...
99997    http://beingremarkable.me/blog/freelancer-or-e...
99998    http://www.300house.com/blog/2011/06/awaken-mo...
99999    http://www.businessinsider.com/infamous-ps3-ip...
Name: url, Length: 93708, dtype: object


In [17]:
'''
The following code gives us:
1. User name
2. Title
3. Length of Title
4. Number of Upvotes
'''

df['by'] = df['by'].fillna('').astype(str)
df['title'] = df['title'].fillna('').astype(str)
df['title_length_chars'] = df['title'].str.len()
df['title_length_words'] = df['title'].str.split().str.len()

In [18]:
'''
Getting a file that has:
- User ('by')
- Title ('title')
- Domain ('domain_name')
- Day of the week ('day_of_week_num')
- Month ('month')
- Hour ('hour')
- Year ('year')
- Time since post ('time_since_post')
- Title length chars ('title_length_chars')
- Score ('score')
'''

selected_columns = ['by', 'title', 'domain_name', 'day_of_week_num', 'month_num', 'hour', 'year', 'time_since_post', 'title_length_chars', 'score']
df_selected = df[selected_columns]
df_selected.to_csv('RelevantDataScrape.csv', index=False)

df['domain_name'].value_counts()
domain_name_counts = df['domain_name'].value_counts()
df['by'].value_counts()
username_counts = df['by'].value_counts()

In [19]:
domaincolumn_names = domain_name_counts.name
print(domaincolumn_names)
usernamecolumn_names = username_counts.name
print(usernamecolumn_names)

count
count


In [20]:
cutoff_value =  1000
filtered_domain_counts = domain_name_counts[domain_name_counts > cutoff_value]
print(filtered_domain_counts)

filtered_username_counts = username_counts[username_counts > cutoff_value]
print(filtered_username_counts)

domain_name
techcrunch.com     1282
www.youtube.com    1220
www.nytimes.com    1066
github.com         1013
Name: count, dtype: int64
Series([], Name: count, dtype: int64)


In [21]:
import numpy as np
import torch # This is where torch is imported
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [22]:
df = pd.read_csv('RelevantDataScrape.csv', low_memory = False)
df = df.dropna()

In [23]:
sentence_vectors = []
for title in tqdm(df['title'], desc="Generating sentence embeddings"):
    with torch.no_grad():
        vector = model.get_sentence_vector(str(title)).cpu().numpy()  # ensure numpy
        sentence_vectors.append(vector)

# Stack into a matrix (shape: num_samples x embedding_dim)
title_embeddings = np.vstack(sentence_vectors)
print(f"[DEBUG] title_embeddings.shape = {title_embeddings.shape}")

Generating sentence embeddings: 100%|█| 93707/93707 [00:02<00:00, 37361.58it/s


[DEBUG] title_embeddings.shape = (93707, 50)


In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

features = df[[
    'score',
    'day_of_week_num',
    'month_num',
    'hour',
    'year',
    'time_since_post',
    'title_length_chars'
]].values

numerical_features = df[[
    'day_of_week_num',
    'month_num',
    'hour',
    'year',
    'time_since_post',
    'title_length_chars']].values
print(f"[DEBUG] Shape of numerical_features: {numerical_features.shape}")

if 'by' in df.columns:
    num_top_users = 100
    top_users = df['by'].value_counts().nlargest(num_top_users).index.tolist()
    print(f"[DEBUG] Identified top {len(top_users)} users for one-hot encoding.")

    # Create 'user_group' column with top users or 'OTHER'
    df['user_group'] = df['by'].apply(lambda x: x if x in top_users else 'OTHER')

    # One-hot encode
    user_one_hot_features = pd.get_dummies(df['user_group'], prefix='user')

    # Debug
    print(f"[DEBUG] User feature columns: {user_one_hot_features.columns.tolist()}")
    print(f"[DEBUG] Shape of user_one_hot_features: {user_one_hot_features.shape}")

    user_one_hot_features_array = user_one_hot_features.values
else:
    user_one_hot_features_array = np.empty((len(df), 0))
    print("[DEBUG] 'by' column not available, skipping user one-hot encoding.")

# --- DOMAINS ---

if 'domain_name' in df.columns:
    num_top_domain = 100
    top_domain = df['domain_name'].value_counts().nlargest(num_top_domain).index.tolist()
    print(f"[DEBUG] Identified top {len(top_domain)} domains for one-hot encoding.")

    # Create 'domain_group' column with top domains or 'OTHER'
    df['domain_group'] = df['domain_name'].apply(lambda x: x if x in top_domain else 'OTHER')

    # One-hot encode
    domain_one_hot_features = pd.get_dummies(df['domain_group'], prefix='domain')

    # Debug
    print(f"[DEBUG] Domain feature columns: {domain_one_hot_features.columns.tolist()}")
    print(f"[DEBUG] Shape of domain_one_hot_features: {domain_one_hot_features.shape}")

    domain_one_hot_features_array = domain_one_hot_features.values
else:
    domain_one_hot_features_array = np.empty((len(df), 0))
    print("[DEBUG] 'domain_name' column not available, skipping domain one-hot encoding.")

# Combine features and target
X = np.hstack((numerical_features, user_one_hot_features_array, domain_one_hot_features_array, title_embeddings))
y = df[['score']].values.reshape(-1, 1)
print(f"[DEBUG] Shape of target variable (y): {y.shape}")

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Shuffle and manually split 80/20
num_samples = X_tensor.shape[0]
indices = torch.randperm(num_samples)
split = int(num_samples * 0.8)

train_indices = indices[:split]
test_indices = indices[split:]

X_train_tensor = X_tensor[train_indices]
y_train_tensor = y_tensor[train_indices]
X_test_tensor = X_tensor[test_indices]
y_test_tensor = y_tensor[test_indices]

# Normalize using training set stats
mean = X_train_tensor.mean(dim=0, keepdim=True)
std = X_train_tensor.std(dim=0, keepdim=True)
std[std == 0] = 1.0  # Prevent division by zero

X_train_tensor = (X_train_tensor - mean) / std
X_test_tensor = (X_test_tensor - mean) / std  # Use train set stats

# Move to device (CPU or GPU)
X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

# Create DataLoaders for batching
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# --- 5. Define the Neural Network Model in PyTorch ---
# Refactored to use nn.Sequential
class HackerNewsPredictor(nn.Module):
    def __init__(self, input_dim):
        super(HackerNewsPredictor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(), # Activation function for the first hidden layer
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(), # Activation function for the second hidden layer
            nn.Dropout(0.3),
            nn.Linear(64, 1) # Output layer with linear activation (default for nn.Linear)
        )

    def forward(self, x):
        return self.network(x)

# Instantiate the model
input_dim = X_train_tensor.shape[1]
model = HackerNewsPredictor(input_dim).to(device) # Move model to device (CPU/GPU)

print("\nPyTorch Neural Network Model Defined:")
print(model)

Using device: cpu
[DEBUG] Shape of numerical_features: (93707, 6)
[DEBUG] Identified top 100 users for one-hot encoding.
[DEBUG] User feature columns: ['user_Anon84', 'user_ColinWright', 'user_DanielRibeiro', 'user_FluidDjango', 'user_Garbage', 'user_J3L2404', 'user_JumpCrisscross', 'user_Libertatea', 'user_MikeCapone', 'user_OTHER', 'user_PatriciaLopes', 'user_RiderOfGiraffes', 'user_Tsiolkovsky', 'user_aaronbrethorst', 'user_abennett', 'user_abraham', 'user_alexandros', 'user_amichail', 'user_anigbrowl', 'user_antr', 'user_apievangelist', 'user_bdfh42', 'user_bensummers', 'user_bjonathan', 'user_bootload', 'user_ca98am79', 'user_canistr', 'user_chaostheory', 'user_cleverjake', 'user_coderdude', 'user_cwan', 'user_danielha', 'user_danso', 'user_danw', 'user_darkduck', 'user_davidw', 'user_denzil_correa', 'user_digiwizard', 'user_duck', 'user_dwynings', 'user_edw519', 'user_evo_9', 'user_fogus', 'user_fromedome', 'user_georgecmu', 'user_gibsonf1', 'user_gnosis', 'user_grellas', 'user_h

In [52]:
# --- 6. Define Loss Function and Optimizer ---
criterion = nn.MSELoss() # Mean Squared Error Loss for regression
optimizer = optim.Adam(model.parameters(), lr=wandb.config['learning_rate']) # Adam optimizer with learning rate

print("\nLoss function (MSE) and Optimizer (Adam) defined.")


Loss function (MSE) and Optimizer (Adam) defined.


In [54]:
# --- 7. Train the Model ---
num_epochs = 100 # Max epochs, EarlyStopping will manage it
patience = wandb.config["epochs"] # Number of epochs to wait for improvement
best_val_loss = float('inf')
epochs_no_improve = 0

print("\nStarting model training...")
for epoch in range(num_epochs):
    model.train() # Set model to training mode
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device) # Move data to device
        optimizer.zero_grad() # Zero the gradients
        outputs = model(inputs) # Forward pass
        loss = criterion(outputs, targets) # Calculate loss
        loss.backward() # Backward pass (compute gradients)
        optimizer.step() # Update weights
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_dataset)

    # Validation phase
    model.eval() # Set model to evaluation mode
    val_running_loss = 0.0
    val_predictions = []
    val_actuals = []
    with torch.no_grad(): # Disable gradient calculations during validation
        for inputs, targets in test_loader: # Using test_loader for simplicity in this example for validation
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            val_loss = criterion(outputs, targets)
            val_running_loss += val_loss.item() * inputs.size(0)
            val_predictions.extend(outputs.cpu().numpy())
            val_actuals.extend(targets.cpu().numpy())
    
    val_epoch_loss = val_running_loss / len(test_dataset) # Use test_dataset for size calculation here

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Val Loss: {val_epoch_loss:.4f}")
    wandb.log({
        "epoch" : epoch + 1,
        "train_loss": epoch_loss,
        "val_loss": val_epoch_loss
    })

    # Early Stopping check
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pth') # Save best model
    else:
        epochs_no_improve += 1
        if epochs_no_improve == patience:
            print(f"Early stopping at epoch {epoch+1} as validation loss did not improve for {patience} epochs.")
            break

print("Model training finished.")




Starting model training...
Epoch [1/100], Train Loss: 1338.2567, Val Loss: 1615.6520
Epoch [2/100], Train Loss: 1326.2476, Val Loss: 1616.1658
Epoch [3/100], Train Loss: 1321.7515, Val Loss: 1616.6325
Epoch [4/100], Train Loss: 1320.7929, Val Loss: 1618.7513
Epoch [5/100], Train Loss: 1316.9074, Val Loss: 1616.6979
Epoch [6/100], Train Loss: 1315.3552, Val Loss: 1614.0039
Epoch [7/100], Train Loss: 1313.6928, Val Loss: 1615.9708
Epoch [8/100], Train Loss: 1310.8122, Val Loss: 1617.7590
Epoch [9/100], Train Loss: 1311.7611, Val Loss: 1616.1585
Epoch [10/100], Train Loss: 1307.9131, Val Loss: 1617.6413
Epoch [11/100], Train Loss: 1305.7236, Val Loss: 1623.5347
Epoch [12/100], Train Loss: 1302.2046, Val Loss: 1616.8567
Epoch [13/100], Train Loss: 1298.4147, Val Loss: 1617.9507
Epoch [14/100], Train Loss: 1292.0043, Val Loss: 1621.3748
Epoch [15/100], Train Loss: 1286.7415, Val Loss: 1621.6066
Epoch [16/100], Train Loss: 1282.1002, Val Loss: 1625.7407
Epoch [17/100], Train Loss: 1274.7641

In [56]:
# --- 8. Evaluate the Model on the Test Set (or load best model and evaluate) ---
model.load_state_dict(torch.load('best_model.pth'))  # Load the best model weights
model.eval()  # Set to evaluation mode

all_predictions = []
all_actuals = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        all_predictions.append(outputs)
        all_actuals.append(targets)

# Concatenate tensors
all_predictions = torch.cat(all_predictions).flatten()
all_actuals = torch.cat(all_actuals).flatten()

# Compute metrics using PyTorch only
mse = torch.mean((all_predictions - all_actuals) ** 2).item()
mae = torch.mean(torch.abs(all_predictions - all_actuals)).item()

# R² calculation: 1 - SSR/SST
ss_res = torch.sum((all_predictions - all_actuals) ** 2)
ss_tot = torch.sum((all_actuals - torch.mean(all_actuals)) ** 2)
r2 = 1 - (ss_res / ss_tot).item()

print(f"\nModel Evaluation on Test Set:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R²): {r2:.2f}")

wandb.log({
    "test_r2": r2,
    "MSE": mse,
    "MAE": mae,
})



Model Evaluation on Test Set:
Mean Squared Error (MSE): 1614.00
Mean Absolute Error (MAE): 13.53
R-squared (R²): 0.01


In [58]:
# --- 9. Make Predictions (Optional) ---
# Prepare a sample for prediction (e.g., the first sample from the test set)
sample_index = 0
sample_input_scaled = X_test_tensor[sample_index:sample_index+1]

model.eval()
with torch.no_grad():
    predicted_upvotes_tensor = model(sample_input_scaled)
    predicted_upvotes = predicted_upvotes_tensor.item()  # Convert to scalar

# Actual value (assuming y_test is a PyTorch tensor or converted here)
actual_upvotes = y_test_tensor[sample_index].item()

print(f"\nPrediction for sample {sample_index}:")
print(f"Actual Upvotes: {actual_upvotes}")
print(f"Predicted Upvotes: {predicted_upvotes:.2f}")


Prediction for sample 0:
Actual Upvotes: 2.0
Predicted Upvotes: 5.39
